In [1]:
"""
The following is an import of PyTorch libraries.
"""
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image

import numpy as np
import random
from data_loader import NEFG3x3Set




Using device: mps


In [3]:


"""
Initialize Hyperparameters
"""
batch_size = 5
learning_rate = 1e-3
num_epochs = 50


"""
Create dataloaders to feed data into the neural network
Default MNIST dataset is used and standard train/test split is performed
"""
dataset = NEFG3x3Set("info_dat_charge.csv",
                     "data/3x12_16_damp00", "dat_charge", transform=True, device=device)
length = len(dataset)
train_split = math.floor(length*.7)
test_split = length - train_split

train_inds, test_inds = torch.utils.data.random_split(
    dataset, [train_split, test_split], generator=torch.Generator().manual_seed(42))
train_data = torch.utils.data.DataLoader(dataset=train_inds, batch_size=5,
                                         shuffle=True)
test_data = torch.utils.data.DataLoader(dataset=test_inds, batch_size=5,
                                        shuffle=True)

"""
Initialize the network and the Adam optimizer
"""
net = VAE().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)


In [4]:


"""
Training the network for a given number of epochs
The loss after every epoch is printed
"""
loss = 0
for epoch in range(num_epochs):
    loss = 0
    for idx, data in enumerate(train_data, 0):
        (inp,_ , imgs, _,_,_, vd, vg) = data
        imgs = imgs.float().to(device)
        vd_fin = torch.empty(imgs.shape).to(device)
        vg_fin = torch.empty(imgs.shape).to(device)

        for j in range(len(vd)):
            vd_fin[j,:,:,:] = vd[j]
            vg_fin[j,:,:,:] = vg[j]

        imgs1 = torch.cat((imgs,vd_fin,vg_fin), dim=1)
    #     # Feeding a batch of images into the network to obtain the output image, mu, and logVar
        out = net(imgs1)
        

    #     # The loss is the BCE loss combined with the KL divergence to ensure the distribution is learnt
    #     # kl_divergence = 0.5 * torch.sum(-1 - logVar + mu.pow(2) + logVar.exp())
    #     loss = F.mse_loss(out, imgs, reduction='sum') #+ kl_divergence
        
    #     # Backpropagation based on the loss
    #     optimizer.zero_grad()
    #     loss.backward()
    #     optimizer.step()
        
    # loss = 0
    
    # net.eval()
    # with torch.no_grad():
    #     for data in test_data:
    #         (inp,_ , imgs, _,_,_, vd, vg) = data
    #         imgs = imgs.float().to(device)
    #         vd_fin = torch.empty(imgs.shape).to(device)
    #         vg_fin = torch.empty(imgs.shape).to(device)

    #         for j in range(len(vd)):
    #             vd_fin[j,:,:,:] = vd[j]
    #             vg_fin[j,:,:,:] = vg[j]

    #         imgs1 = torch.cat((imgs,vd_fin,vg_fin), dim=1)
                    
    #         out = net(imgs1)
    #         # kl_divergence = 0.5 * torch.sum(-1 - logVAR + mu.pow(2) + logVAR.exp())
    #         loss += F.mse_loss(out, imgs, reduction='sum')# + kl_divergence
        
    #     loss =loss/len(test_data)
    #     print('Epoch {}: Loss {}'.format(epoch,loss))
        

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random

net.eval()
with torch.no_grad():
    for data in test_data:
        (inp, cmp, imgs, _,_,_, vd, vg) = data
        imgs = imgs.float().to(device)
        vd_fin = torch.empty(imgs.shape).to(device)
        vg_fin = torch.empty(imgs.shape).to(device)

        for j in range(len(vd)):
            vd_fin[j,:,:,:] = vd[j]
            vg_fin[j,:,:,:] = vg[j]

        imgs1 = torch.cat((imgs,vd_fin,vg_fin), dim=1)
        
        cmp = cmp.float().to(device)
        
        plt.subplot(121)
        a = plt.imshow(np.squeeze(imgs[0].cpu()))
        c_bar = plt.colorbar(a)
        out, mu, logVAR = net(imgs1)
        outimg = out[0].cpu()
        plt.subplot(122)
        a = plt.imshow(np.squeeze(outimg))
        c_bar = plt.colorbar(a)
        print(F.mse_loss(imgs[0], out[0], reduction='sum'))
        print(F.mse_loss(imgs[0], cmp[0], reduction='sum'))
        
        break

In [ ]:
import matplotlib.pyplot as plt

for data in dataset:
    (inp, cmp, imgs, _,_,_, vd, vg) = data
    imgs = imgs.float().to("cpu")
    print(imgs[0])
    a = plt.imshow(imgs[0].T)
    plt.colorbar(a)
    plt.show()
    break
  